In [1]:
# @title 🟢 Some configs

Sematic_Model = "sentence-transformers/all-MiniLM-L6-v2" # @param {"type": "string"}

Ollama_Model = "llama3.2:latest" # @param {"type": "string"}

Use_Qdrant_Local = True # @param {"type": "boolean"}

Qdrant_Cloud_Url = "" # @param {"type": "string"}
Qdrant_Cloud_Api_Key = "" # @param {"type": "string"}

In [2]:
import threading
import asyncio

async def __run_process(command):
    print('>>> starting', *command)
    process = await asyncio.create_subprocess_exec(
        *command,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )
    async def __pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            __pipe(process.stdout),
            __pipe(process.stderr),
        )
    await asyncio.gather(__pipe(process.stdout), __pipe(process.stderr))

def __run_command(command):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(__run_process(command))
    loop.close()

def run_command(command):
    threading.Thread(target=__run_command, args=(command,)).start()

In [3]:
# Initialize Qdrant
from qdrant_client import QdrantClient
import subprocess
import time

if Use_Qdrant_Local:
    # Install udocker
    !pip install udocker -qU

    # Run qdrant locally
    run_command(["udocker", "--allow-root", "run", "-p", "6333:6333", "qdrant/qdrant:latest"])

    # Waiting for qdrant start
    while True:
        try:
            result = subprocess.run(["curl", "-L", "http://localhost:6333"], capture_output=True, text=True)

            # Check if the curl command was successful
            if result.returncode == 0:
                print("Qdrant is up and running!")
                break
            else:
                print("Waiting for Qdrant to start...")
                time.sleep(1)
        except:
            time.sleep(1)

    qdrant = QdrantClient(
        url="http://localhost:6333"
    )
else:
    qdrant = QdrantClient(
        url=Qdrant_Cloud_Url,
        api_key=Qdrant_Cloud_Api_Key
    )

C:\Users\thain\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception in thread Thread-5 (__run_command):
Traceback (most recent call last):
  File "c:\Users\thain\anaconda3\envs\RAG\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\thain\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\thain\anaconda3\envs\RAG\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\thain\AppData\Local\Temp\ipykernel_35216\393623484.py", line 24, in __run_command
  File "c:\Users\thain\anaconda3\envs\RAG\Lib\asyncio\base_events.py", line 687, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "C:\Us

>>> starting udocker --allow-root run -p 6333:6333 qdrant/qdrant:latest
Qdrant is up and running!


In [4]:
import uuid
from qdrant_client.http import models
from qdrant_client.models import PointStruct


DEFAULT_VECTOR_SIZE = 768
DEFAULT_DISTANCE = "cosine"


class QdrantProvider:
    def __init__(self):
        # Initialize the QdrantProvider with a specific collection name
        self.vector_size = DEFAULT_VECTOR_SIZE
        self.distance = DEFAULT_DISTANCE

    def create_collection(self, collection_name: str):
        # Check if the collection already exists
        if collection_name in self.list_collections():
            print(f"Collection `{collection_name}` already exists.")
            return

        # Create a new collection with the specified vector size and distance metric
        qdrant.create_collection(
            collection_name=collection_name,
            vectors_config=models.VectorParams(
                size=self.vector_size,
                distance=models.Distance[self.distance.upper()]
            )
        )
        print(f"Collection created `{collection_name}`")
        

    def list_collections(self):
        # List all existing collections in the Qdrant database
        collections = qdrant.get_collections()
        return [col.name for col in collections.collections]

    def drop_collection(self, collection_name: str):
        # Drop (delete) the specified collection from Qdrant
        qdrant.delete_collection(collection_name)
        print(f"Collection dropped `{collection_name}`")

    def add_vectors_(self, collection_name: str, semantic_chunks):
        """Add multiple vectors to the Qdrant collection."""
        points = []
        for i, chunk in enumerate(semantic_chunks):
            chunk_text = " ".join(chunk)
            # if len(chunk_text.split()) < 5:
            #     continue
            vector = embedding(chunk_text)
            point = PointStruct(
                id=str(uuid.uuid4()),  # Unique identifier for the vector
                vector=vector.tolist(),  # Convert numpy array to list
                payload={
                    "content": chunk_text
                }
            )
            points.append(point)
        qdrant.upsert(collection_name=collection_name, points=points)
        print(f"{len(points)} Vectors added to `{collection_name}`")

    def search_vector(self, collection_name: str, vector: list[float], limit=3, with_payload=True):
        # Perform the search query in Qdrant with the provided parameters
        search_result = qdrant.search(
            collection_name=collection_name,
            query_vector=vector,
            limit=limit,  # Limit the number of search results
            with_payload=with_payload,  # Whether to include payload in results
        )
        print(f"Vector searched `{collection_name}`")
        return search_result

    def get_all_vectors(self, collection_name: str):
        # Retrieve all vectors from the collection by scrolling through all data
        all_points = []
        has_more = True
        offset = None

        while has_more:
            points, next_offset = qdrant.scroll(
                collection_name=collection_name,
                offset=offset,  # Start from the current offset
                limit=100  # Retrieve 100 vectors at a time (adjustable)
            )

            all_points.extend(points)  # Append the points to the list
            offset = next_offset  # Update the offset for the next scroll

            if next_offset is None:
                has_more = False  # Stop when no more points are available

        print(f"All vectors retrieved `{collection_name}`")
        return all_points


    def delete_all_vectors(self, collection_name: str):
        # Delete all vectors from the collection by deleting the collection itself
        qdrant.delete_collection(collection_name)

        # Recreate the collection after deletion to keep it available
        self.create_collection(collection_name)
        print("All vectors deleted successfully.")



In [5]:
# from io import BytesIO
# from PyPDF2 import PdfReader
# import re

# def extract_and_split_sentences(file_content: BytesIO, file_name: str) -> list:
#     """
#     Extracts text content from a PDF file and splits it into sentences.
#     """
#     if not isinstance(file_content, BytesIO):
#         file_content = BytesIO(file_content)
    
#     file_content.seek(0)
    
#     if not file_name.lower().endswith('.pdf'):
#         raise ValueError(f"Unsupported file format: {file_name}. Only PDF is supported.")
    
#     try:
#         reader = PdfReader(file_content)
#         content = ""
#         for page_number, page in enumerate(reader.pages):
#             try:
#                 page_text = page.extract_text()
#                 if page_text:
#                     content += page_text + "\n"
#                 else:
#                     print(f"Warning: Page {page_number + 1} could not be read properly.")
#             except Exception as e:
#                 print(f"Error extracting text from page {page_number + 1}: {e}")
#     except Exception as e:
#         raise ValueError(f"Failed to read the PDF file: {e}")
    
#     content = content.replace("\n", " ").strip()
#     content = re.sub(r'\s+', ' ', content)  # Remove multiple spaces
#     sentences = re.split(r'(?<=[.?!])\s+', content)
#     return sentences

In [6]:
from io import BytesIO
from PyPDF2 import PdfReader
from docx import Document
import re

def extract_and_split_sentences(file_content: BytesIO, file_name: str) -> list:
    """
    Extracts text content from a file (PDF, TXT, DOCX) and splits it into sentences.
    """
    if not isinstance(file_content, BytesIO):
        file_content = BytesIO(file_content)
    
    file_content.seek(0)
    content = ""

    # Determine file type and extract content
    if file_name.lower().endswith('.pdf'):
        # Handle PDF file
        try:
            reader = PdfReader(file_content)
            for page_number, page in enumerate(reader.pages):
                try:
                    page_text = page.extract_text()
                    if page_text:
                        content += page_text + "\n"
                    else:
                        print(f"Warning: Page {page_number + 1} could not be read properly.")
                except Exception as e:
                    print(f"Error extracting text from page {page_number + 1}: {e}")
        except Exception as e:
            raise ValueError(f"Failed to read the PDF file: {e}")

    elif file_name.lower().endswith('.txt'):
        # Handle TXT file
        try:
            content = file_content.read().decode('utf-8')  # Decode bytes to string
        except Exception as e:
            raise ValueError(f"Failed to read the TXT file: {e}")

    elif file_name.lower().endswith('.docx'):
        # Handle DOCX file
        try:
            document = Document(file_content)
            for paragraph in document.paragraphs:
                content += paragraph.text + "\n"
        except Exception as e:
            raise ValueError(f"Failed to read the DOCX file: {e}")

    else:
        raise ValueError(f"Unsupported file format: {file_name}. Only PDF, TXT, and DOCX are supported.")
    
    # Normalize spaces and clean content
    content = re.sub(r'\s+', ' ', content).strip()  # Remove multiple spaces
    sentences = re.split(r'(?<=[.?!])\s+', content)  # Split into sentences
    return sentences


In [7]:
file_content = BytesIO(open('bo_luat_dan_su.txt', 'rb').read())
extracted_text = extract_and_split_sentences(file_content, 'luatdansu.txt')
extracted_text

['QUỐC HỘI -------- CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc --------------- Luật số: 91/2015/QH13 Hà Nội, ngày 24 tháng 11 năm 2015 BỘ LUẬT DÂN SỰ Căn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam ; Quốc hội ban hành Bộ luật dân sự.',
 'Phần thứ nhất QUY ĐỊNH CHUNG Chương I NHỮNG QUY ĐỊNH CHUNG Điều 1.',
 'Phạm vi điều chỉnh Bộ luật này quy định địa vị pháp lý, chuẩn mực pháp lý về cách ứng xử của cá nhân, pháp nhân; quyền, nghĩa vụ về nhân thân và tài sản của cá nhân, pháp nhân trong các quan hệ được hình thành trên cơ sở bình đẳng, tự do ý chí, độc lập về tài sản và tự chịu trách nhiệm (sau đây gọi chung là quan hệ dân sự).',
 'Điều 2.',
 'Công nhận, tôn trọng, bảo vệ và bảo đảm quyền dân sự 1.',
 'Ở nước Cộng hòa xã hội chủ nghĩa Việt Nam, các quyền dân sự được công nhận, tôn trọng, bảo vệ và bảo đảm theo Hiến pháp và pháp luật.',
 '2.',
 'Quyền dân sự chỉ có thể bị hạn chế theo quy định của luật trong trường hợp cần thiết vì lý do quốc phòng, an ninh q

In [8]:
# file_content = BytesIO(open('ICAMCS2024_1DCNN_Imputation.pdf', 'rb').read())
# extracted_text = extract_and_split_sentences(file_content, 'AI-INFRASTRUCTURE.pdf')
# extracted_text

In [9]:
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import BertTokenizer, BertModel

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


def create_semantic_chunks(sentences):
    semantic_chunks = []
    sen_embeddings = []

    # Embed sentences and move embeddings to GPU
    for sentence in sentences:
        with torch.no_grad():  # Disable gradient calculation for inference
            # Tokenize and encode the sentence
            inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
            outputs = model(**inputs)

            # Extract the [CLS] token embedding
            emb = outputs.last_hidden_state[:, 0, :]  # Shape: (1, hidden_size)
        sen_embeddings.append(emb.cpu().numpy())  # Move embedding back to CPU and store as NumPy array

    # Compare cosine similarity to create chunks
    for i in range(len(sentences)):
        if i == 0:
            semantic_chunks.append([sentences[i]])
        else:
            # Compute cosine similarity between current and previous sentence embeddings
            similarity_score = cosine_similarity(sen_embeddings[i-1], sen_embeddings[i])
            if similarity_score[0][0] > 0.95:  # Adjust threshold as needed
                semantic_chunks[-1].append(sentences[i])
            else:
                semantic_chunks.append([sentences[i]])

    return semantic_chunks

def embedding(chunk):
    inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        # Sử dụng pooler_output làm embedding (hoặc hidden_states[0] nếu cần toàn bộ token embeddings)
        embedding = outputs.pooler_output.squeeze(0)  # (1, hidden_size) → (hidden_size)

    return embedding.cpu().numpy()


from sentence_transformers import SentenceTransformer

# Load a pre-trained SentenceTransformer model
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')  


def embedding(chunk):
    # Generate the embedding for a given chunk using SentenceTransformer
    embedding = model.encode(chunk, convert_to_numpy=True, device=device)
    return embedding

def create_semantic_chunks(sentences):
    semantic_chunks = []
    embeddings = model.encode(sentences, convert_to_numpy=True, device=device)  # Embed all sentences

    for i, sentence in enumerate(sentences):
        if i == 0:
            semantic_chunks.append([sentence])
        else:
            # Compute cosine similarity between current and previous embeddings
            similarity_score = cosine_similarity([embeddings[i-1]], [embeddings[i]])[0][0]
            if similarity_score > 0.7:  # Adjust threshold as needed
                semantic_chunks[-1].append(sentence)
            else:
                semantic_chunks.append([sentence])

    return semantic_chunks



Using device: cuda



In [10]:
semantic_chunks = create_semantic_chunks(extracted_text) 
semantic_chunks, len(semantic_chunks)

([['QUỐC HỘI -------- CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc --------------- Luật số: 91/2015/QH13 Hà Nội, ngày 24 tháng 11 năm 2015 BỘ LUẬT DÂN SỰ Căn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam ; Quốc hội ban hành Bộ luật dân sự.'],
  ['Phần thứ nhất QUY ĐỊNH CHUNG Chương I NHỮNG QUY ĐỊNH CHUNG Điều 1.',
   'Phạm vi điều chỉnh Bộ luật này quy định địa vị pháp lý, chuẩn mực pháp lý về cách ứng xử của cá nhân, pháp nhân; quyền, nghĩa vụ về nhân thân và tài sản của cá nhân, pháp nhân trong các quan hệ được hình thành trên cơ sở bình đẳng, tự do ý chí, độc lập về tài sản và tự chịu trách nhiệm (sau đây gọi chung là quan hệ dân sự).'],
  ['Điều 2.'],
  ['Công nhận, tôn trọng, bảo vệ và bảo đảm quyền dân sự 1.',
   'Ở nước Cộng hòa xã hội chủ nghĩa Việt Nam, các quyền dân sự được công nhận, tôn trọng, bảo vệ và bảo đảm theo Hiến pháp và pháp luật.'],
  ['2.'],
  ['Quyền dân sự chỉ có thể bị hạn chế theo quy định của luật trong trường hợp cần thiết vì lý do 

In [11]:
qdrant_name = 'VectorDBforLAW'

vectordb_provider = QdrantProvider()
vectordb_provider.create_collection(qdrant_name)
# vectordb_provider.delete_all_vectors(qdrant_name)
# vectordb_provider.get_all_vectors(qdrant_name)


Collection `VectorDBforLAW` already exists.


In [12]:
vectordb_provider.add_vectors_(qdrant_name, semantic_chunks)

877 Vectors added to `VectorDBforLAW`


In [13]:
vectordb_provider.get_all_vectors(qdrant_name)

All vectors retrieved `VectorDBforLAW`


[Record(id='000d680d-9a89-4458-b936-18777c4b61b3', payload={'content': '3.'}, vector=None, shard_key=None, order_value=None),
 Record(id='003bef70-6960-49ff-8da5-a67e3617382f', payload={'content': 'Trường hợp chấm dứt việc giám hộ quy định tại điểm c và điểm d khoản 1 Điều 62 của Bộ luật này thì trong thời hạn 15 ngày, kể từ ngày chấm dứt việc giám hộ, người giám hộ thanh toán tài sản và chuyển giao quyền, nghĩa vụ phát sinh từ giao dịch dân sự vì lợi ích của người được giám hộ cho cha, mẹ của người được giám hộ.'}, vector=None, shard_key=None, order_value=None),
 Record(id='0088638e-cff3-4d57-8635-9c8c27cda756', payload={'content': 'Giao dịch dân sự vô hiệu Giao dịch dân sự không có một trong các điều kiện được quy định tại Điều 117 của Bộ luật này thì vô hiệu, trừ trường hợp Bộ luật này có quy định khác.'}, vector=None, shard_key=None, order_value=None),
 Record(id='009af980-44a1-41af-a473-3a105a3ba7a8', payload={'content': 'Công nhận, tôn trọng, bảo vệ và bảo đảm quyền dân sự 1.'}, 

In [20]:
question = """Chương III phần cá nhân
Mục 3. 
NƠI CƯ TRÚ
Điều 40. 
Nơi cư trú của cá nhân bao gồm những gì?"""
question_vector = embedding(question)

answer = vectordb_provider.search_vector(qdrant_name, question_vector, limit=5)
answer

Vector searched `VectorDBforLAW`


[ScoredPoint(id='1e4c4171-8b3f-4036-a2cf-29b5b75d2ebf', version=1, score=0.7952238, payload={'content': 'Nơi cư trú của sĩ quan quân đội, quân nhân chuyên nghiệp, công nhân, viên chức quốc phòng là nơi đơn vị của người đó đóng quân, trừ trường hợp họ có nơi cư trú theo quy định tại khoản 1 Ðiều 40 của Bộ luật này .'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='2f0aee56-9092-4fc0-b86f-41fd2e8eef57', version=2, score=0.7952238, payload={'content': 'Nơi cư trú của sĩ quan quân đội, quân nhân chuyên nghiệp, công nhân, viên chức quốc phòng là nơi đơn vị của người đó đóng quân, trừ trường hợp họ có nơi cư trú theo quy định tại khoản 1 Ðiều 40 của Bộ luật này .'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='52cb61a4-5bfb-44c9-83e1-7bdfc0519567', version=0, score=0.7952238, payload={'content': 'Nơi cư trú của sĩ quan quân đội, quân nhân chuyên nghiệp, công nhân, viên chức quốc phòng là nơi đơn vị của người đó đóng quân, trừ trường hợp họ có nơi cư trú

In [15]:
from rank_bm25 import BM25Okapi

def query_qdrant(question: str, collection_name=qdrant_name, top_k=5):
    """
    Queries Qdrant to retrieve the most relevant chunks based on the user question.
    """
    # Embed the question using BERT uncased
    question_vector = embedding(question)
    
    try:
        search_results = vectordb_provider.search_vector(
            collection_name=collection_name,
            vector=question_vector,
            limit=top_k,  # Number of top results to return
            with_payload=True  # Return payloads (metadata)
        )
    except Exception as e:
        print(f"Failed to query Qdrant: {e}")
        return []

    top_results = []
    for result in search_results:
        content = result.payload.get("content", "No content available")
        score = result.score
        
        top_results.append({
            "content": content,
            "score": score
        })
    
    # Re-rank the results using BM25
    ranked_results = bm25_rerank(question, top_results)
    
    return ranked_results


def bm25_rerank(question, results):
    """
    Re-rank the results using BM25.
    """
    # Tokenize the content of each chunk for BM25
    tokenized_corpus = [result['content'].split() for result in results]
    bm25 = BM25Okapi(tokenized_corpus)
    
    # Tokenize the question
    question_tokens = question.split()
    
    # Get BM25 scores
    scores = bm25.get_scores(question_tokens)
    
    # Sort results by BM25 score
    sorted_results = sorted(zip(results, scores), key=lambda x: x[1], reverse=True)
    
    # Return the sorted results
    return [item[0] for item in sorted_results]

In [19]:
question = """Chương III phần cá nhân
Mục 3. 
NƠI CƯ TRÚ
Điều 40. 
Nơi cư trú của cá nhân bao gồm những gì?"""

# Truy vấn Qdrant để lấy câu trả lời
results = query_qdrant(question, collection_name=qdrant_name, top_k=5)

if results:
    print(f"Top {len(results)} results from Qdrant:")
    for i, result in enumerate(results):
        print(f"Result {i+1}:")
        print(f"Score: {result['score']:.4f}")
        print(f"Content: {result['content']}\n")
else:
    print("No results found.")

results[0]['content']

Vector searched `VectorDBforLAW`
Top 5 results from Qdrant:
Result 1:
Score: 0.7816
Content: Chương III CÁ NHÂN Mục 1.

Result 2:
Score: 0.7816
Content: Chương III CÁ NHÂN Mục 1.

Result 3:
Score: 0.7952
Content: Nơi cư trú của sĩ quan quân đội, quân nhân chuyên nghiệp, công nhân, viên chức quốc phòng là nơi đơn vị của người đó đóng quân, trừ trường hợp họ có nơi cư trú theo quy định tại khoản 1 Ðiều 40 của Bộ luật này .

Result 4:
Score: 0.7952
Content: Nơi cư trú của sĩ quan quân đội, quân nhân chuyên nghiệp, công nhân, viên chức quốc phòng là nơi đơn vị của người đó đóng quân, trừ trường hợp họ có nơi cư trú theo quy định tại khoản 1 Ðiều 40 của Bộ luật này .

Result 5:
Score: 0.7952
Content: Nơi cư trú của sĩ quan quân đội, quân nhân chuyên nghiệp, công nhân, viên chức quốc phòng là nơi đơn vị của người đó đóng quân, trừ trường hợp họ có nơi cư trú theo quy định tại khoản 1 Ðiều 40 của Bộ luật này .



'Chương III CÁ NHÂN Mục 1.'